### Simulations

This file includes the code for generating the simulated data. Each data file will contain a set of 5000 exposures randomly sampled without replacement from the Utah vital records cohort, the synthetic confounder, and the synthetic outcome. Indicator variables are also included to describe the critical window's structure.  

This runs 600 simulations for interactions within a moderate, smooth critical window in the middle of the 20-week period.

#### Outside of loop

In [5]:
import os
import numpy as np
import pandas as pd
import seaborn as sns
import scipy.stats as stats
import multiprocessing as mp
import statsmodels.api as sm
from scipy.stats import norm
import matplotlib.pyplot as plt
from econml.dml import LinearDML
from econml.dml import CausalForestDML
from sklearn.preprocessing import PolynomialFeatures

np.set_printoptions(suppress=True)

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


##### Critical window coefficient creation

In [6]:
study_period = np.linspace(1, 20, 20)
effect_size = -15 # total reduction of 15 percentiles (based on the average effect of a 200-gram reduction in birthweight, given Sun et al)
# the window should be 9 wide, then 7 wide, then 3 wide
window_start = 6
window_center = 10
window_end = 14

uni_pdf = stats.Uniform(a = window_start, b = window_end)
uni_wide_fx  = uni_pdf.pdf(study_period) * effect_size

uni_pdf = stats.Uniform(a = window_start + 1, b = window_end - 1)
uni_moderate_fx  = uni_pdf.pdf(study_period) * effect_size

uni_pdf = stats.Uniform(a = window_start + 3, b = window_end - 3)
uni_narrow_fx  = uni_pdf.pdf(study_period) * effect_size

norm_pdf = stats.norm.pdf(study_period, window_center, 2.5)
norm_wide_fx  = norm_pdf * effect_size 

norm_pdf = stats.norm.pdf(study_period, window_center, 1)
norm_moderate_fx  = norm_pdf * effect_size 

norm_pdf = stats.norm.pdf(study_period, window_center, 0.5)
norm_narrow_fx = norm_pdf * effect_size

In [7]:
(norm_moderate_fx[8] + norm_moderate_fx[9]) * 1.25

# intx_effect = (norm_moderate_fx[8] + norm_moderate_fx[9]) * 1.25

-12.017118842260802

In [8]:
# combinations to be simulated
cw_coefs = pd.DataFrame({#"norm_wide_fx": norm_wide_fx, 
              "norm_moderate_fx": norm_moderate_fx})#, 
              #"norm_narrow_fx": norm_narrow_fx,
              #"uni_wide_fx": uni_wide_fx, 
              #"uni_moderate_fx": uni_moderate_fx, 
              #"uni_narrow_fx": uni_narrow_fx})

cw_combos = pd.DataFrame({"coefficients": [cw_coefs["norm_moderate_fx"]],#, 
                                           #cw_coefs["norm_narrow_fx"],
                                           #cw_coefs["uni_wide_fx"], 
                                           #cw_coefs["uni_moderate_fx"], 
                                           #cw_coefs["uni_narrow_fx"]],
                          "sizes": [#"wide", 
                                    "moderate"],
                                    #, "narrow",
                                    #"wide", "moderate", "narrow"],
                          "times": [#"smooth", 
                                    "smooth"]})
                                    #, "smooth",
                                    #"naive", "naive", "naive"]})

# save, for plotting
cw_coefs.to_csv("data/" + "true_intx_cw_fx_" + str(effect_size) + ".csv", sep = ',', index = False)

##### Reading in the data

In [9]:
births = pd.read_csv("data/birth_w_percentile_confounders.csv")
births.head()

o3_mean = births[['max_o3_01', 'max_o3_02', 'max_o3_03', 'max_o3_04', 
                  'max_o3_05', 'max_o3_06', 'max_o3_07', 'max_o3_08', 
                  'max_o3_09', 'max_o3_10', 'max_o3_11', 'max_o3_12', 
                  'max_o3_13', 'max_o3_14', 'max_o3_15', 'max_o3_16', 
                  'max_o3_17', 'max_o3_18', 'max_o3_19', 'max_o3_20']].values.mean()
o3_sd = births[['max_o3_01', 'max_o3_02', 'max_o3_03', 'max_o3_04', 
                'max_o3_05', 'max_o3_06', 'max_o3_07', 'max_o3_08', 
                'max_o3_09', 'max_o3_10', 'max_o3_11', 'max_o3_12', 
                'max_o3_13', 'max_o3_14', 'max_o3_15', 'max_o3_16', 
                'max_o3_17', 'max_o3_18', 'max_o3_19', 'max_o3_20']].values.std()

bwp_mean = births["bw_percentile"].mean()
bwp_sd = births["bw_percentile"].std()


/var/folders/75/b6cym0892z93rs6dbfy789240000gn/T/ipykernel_445/2057111297.py:1: DtypeWarning: Columns (15,23) have mixed types. Specify dtype option on import or set low_memory=False.
  births = pd.read_csv("data/birth_w_percentile_confounders.csv")


#### Loop

In [10]:
# def sim_function(iteration, cw_window_type, window_size, window_time):
        
#     ## collecting exposure data

#     n_samples = 5000

#     n_X = 1
#     n_T = 20
#     n_W = 1

#     # treatments / exposures; 10 ppb scale
#     T_sample = births.sample(n = n_samples, replace = False)

#     T_01 = (T_sample['max_o3_01'] - o3_mean) / 10
#     T_03 = (T_sample['max_o3_03'] - o3_mean) / 10
#     T_02 = (T_sample['max_o3_02'] - o3_mean) / 10
#     T_04 = (T_sample['max_o3_04'] - o3_mean) / 10
#     T_05 = (T_sample['max_o3_05'] - o3_mean) / 10
#     T_06 = (T_sample['max_o3_06'] - o3_mean) / 10
#     T_07 = (T_sample['max_o3_07'] - o3_mean) / 10
#     T_08 = (T_sample['max_o3_08'] - o3_mean) / 10
#     T_09 = (T_sample['max_o3_09'] - o3_mean) / 10
#     T_10 = (T_sample['max_o3_10'] - o3_mean) / 10
#     T_11 = (T_sample['max_o3_11'] - o3_mean) / 10
#     T_12 = (T_sample['max_o3_12'] - o3_mean) / 10
#     T_13 = (T_sample['max_o3_13'] - o3_mean) / 10
#     T_14 = (T_sample['max_o3_14'] - o3_mean) / 10
#     T_15 = (T_sample['max_o3_15'] - o3_mean) / 10
#     T_16 = (T_sample['max_o3_16'] - o3_mean) / 10
#     T_17 = (T_sample['max_o3_17'] - o3_mean) / 10
#     T_18 = (T_sample['max_o3_18'] - o3_mean) / 10
#     T_19 = (T_sample['max_o3_19'] - o3_mean) / 10
#     T_20 = (T_sample['max_o3_20'] - o3_mean) / 10

#     # stack
#     T_vars = np.vstack((T_01, T_02, T_03, T_04, T_05,
#                         T_06, T_07, T_08, T_09, T_10,
#                         T_11, T_12, T_13, T_14, T_15,
#                         T_16, T_17, T_18, T_19, T_20))

#     ## creating confounder coefficient

#     # we're only confounding the critical exposures if the effect is < -5 (can be changed later)
#     critical_to_confound = [index for index, value in enumerate(cw_window_type) if value < -0.5]

#     X = [sum(fx) for fx in zip(*T_vars[critical_to_confound]*1.25)] + np.random.normal(size = n_samples, loc = 0, scale = 1)
#     # z scale
#     X = X / X.std()
#     X = pd.Series(X)
#     X = X.to_numpy()

#     ## creating the outcome

#     # the coefficients and treatment variables
#     tx_fx, tx = cw_window_type, T_vars
#     # the vectors of each coefficient * treatment variable
#     tx_fx_list = [tx[i] * tx_fx[i] for i in np.arange(0, 20)]
#     # tx_fx_list = [tx[10] * tx_fx[10]]
#     # total treatment effect by individual
#     total_tx_fx = [sum(fx) for fx in zip(*tx_fx_list)]

#     # the confounder effect (z-scaled, see above)
#     b_W0y = 1.5
#     # the vector of confounder effects
#     wx_fx = X * b_W0y

#     # bit of noise
#     e = np.random.normal(size=n_samples, loc = 0, scale = 1)

#     b_int = bwp_mean

#     # interaction #######
#     b_intx = -12 #(norm_moderate_fx[8] + norm_moderate_fx[9]) * 1.25
#     intx_fx = b_intx * tx[8] * tx[9] #[b_intx * tx[i] for i in np.arange(8, 10)]

#     y = b_int + total_tx_fx + wx_fx + e + intx_fx

#     # adding an indicator for critical / not
#     # critical_ones = [index for index, value in enumerate(cw_window_type) if value < -10]

#     # critical_or_not = []

#     # for i in range(0, 20):
#     #     if i in critical_ones:
#     #         critical_or_not.append("critical")
#     #     else:
#     #         critical_or_not.append("not critical")

#     ## store the data file

#     sim_dat = pd.DataFrame({'sim_index': iteration, 'cw_size': window_size,
#                             'cw_timedep': window_time, #"critical": critical_or_not,
#                             'total_fx_size': effect_size,
#                             'y_hat': y, 'true_y': T_sample["birthweightgrams"], 'x': X,
#                             'tx_01': T_01, 'tx_02': T_02, 'tx_03': T_03, 'tx_04': T_04, 
#                             'tx_05': T_05, 'tx_06': T_06, 'tx_07': T_07, 'tx_08': T_08, 
#                             'tx_09': T_09, 'tx_10': T_10, 'tx_11': T_11, 'tx_12': T_12, 
#                             'tx_13': T_13, 'tx_14': T_14, 'tx_15': T_15, 'tx_16': T_16, 
#                             'tx_17': T_17, 'tx_18': T_18, 'tx_19': T_19, 'tx_20': T_20})

#     sim_dat.to_csv("data/intx_sim_data/" + "sim" + str(iteration).zfill(3) + "_" + window_size + "_" + window_time + ".csv", sep = ',', index = False)

In [11]:
# nsims = 2

# ## one cw structure
# # [sim_function(x) for x in range(1, nsims+1)]

# [sim_function(x, cw_combos.loc[0, "coefficients"], 
#         cw_combos.loc[0, "sizes"], 
#         cw_combos.loc[0, "times"]) for x in range(1, nsims+1)]


# # with mp.Pool(mp.cpu_count()) as p:
# #     p.map([sim_function(x, cw_combos.loc[1, "coefficients"], 
# #                         cw_combos.loc[1, "sizes"], 
# #                         cw_combos.loc[1, "times"]) for x in range(1, nsims+1)])
# # ## all cw structures
# # for i in range(0, len(cw_combos)-1):
# #     [sim_function(x, cw_combos.loc[i, "coefficients"], 
# #              cw_combos.loc[i, "sizes"], 
# #              cw_combos.loc[i, "times"]) for x in range(1, nsims+1)]


[None, None]

#### QC
Ran LinearDML to make sure one simulation gives expected result

In [12]:
# T = pd.DataFrame({'noncritical_tx_1': T_01, 'noncritical_tx_2': T_02, 
#                   'noncritical_tx_3': T_03, 'noncritical_tx_4': T_04, 
#                   'noncritical_tx_5': T_05, 'noncritical_tx_6': T_06, 
#                   'noncritical_tx_7': T_07, 'critical_tx_8': T_08, 
#                   'critical_tx_9': T_09, 'critical_tx_10': T_10,
#                   'critical_tx_11': T_11, 'critical_tx_12': T_12, 
#                   'noncritical_tx_13': T_13, 'noncritical_tx_14': T_14, 
#                   'noncritical_tx_15': T_15, 'noncritical_tx_16': T_16, 
#                   'noncritical_tx_17': T_17, 'noncritical_tx_18': T_18, 
#                   'noncritical_tx_19': T_19, 'noncritical_tx_20': T_20})
# X = pd.DataFrame(X, columns=['confounder'])
# y = pd.DataFrame({'birthweight': y})

# model_y = 'linear'
# model_t = 'linear'

# # T = pd.DataFrame({"tx_10": tx[10]})

# est = LinearDML(model_y=model_y, model_t=model_t,
#                 discrete_treatment=False) 

# est.fit(y, T=T, W=X, X=X)

# est.marginal_ate_inference(T, X)

# ### seems to struggle with collinearity
# ### - sometimes adjacent time steps are significant
# ### - sometimes far time steps are significant

### Applying simulated data to causal RF  

This bit runs the causal RF with each simulated data file and stores the results.  
Question: should we compare the causal random forest to the LinearDML?

In [13]:
# # Note: sim number is stored in the csv, in case we want additional comparisons later
# sim_files = os.listdir("data/intx_sim_data/")

# sim_csvs = []

# for file_name in sim_files:
#     if file_name.endswith('.csv'):
#         sim_csvs.append(file_name)

# sim_csvs.sort()

In [49]:
def dml_function(iteration, cw_window_type, window_size, window_time):

    #### Part I: data generation
    n_samples = 5000

    n_X = 1
    n_T = 20
    n_W = 1

    # treatments / exposures; 10 ppb scale
    T_sample = births.sample(n = n_samples, replace = False)

    T_01 = (T_sample['max_o3_01'] - o3_mean) / 10
    T_03 = (T_sample['max_o3_03'] - o3_mean) / 10
    T_02 = (T_sample['max_o3_02'] - o3_mean) / 10
    T_04 = (T_sample['max_o3_04'] - o3_mean) / 10
    T_05 = (T_sample['max_o3_05'] - o3_mean) / 10
    T_06 = (T_sample['max_o3_06'] - o3_mean) / 10
    T_07 = (T_sample['max_o3_07'] - o3_mean) / 10
    T_08 = (T_sample['max_o3_08'] - o3_mean) / 10
    T_09 = (T_sample['max_o3_09'] - o3_mean) / 10
    T_10 = (T_sample['max_o3_10'] - o3_mean) / 10
    T_11 = (T_sample['max_o3_11'] - o3_mean) / 10
    T_12 = (T_sample['max_o3_12'] - o3_mean) / 10
    T_13 = (T_sample['max_o3_13'] - o3_mean) / 10
    T_14 = (T_sample['max_o3_14'] - o3_mean) / 10
    T_15 = (T_sample['max_o3_15'] - o3_mean) / 10
    T_16 = (T_sample['max_o3_16'] - o3_mean) / 10
    T_17 = (T_sample['max_o3_17'] - o3_mean) / 10
    T_18 = (T_sample['max_o3_18'] - o3_mean) / 10
    T_19 = (T_sample['max_o3_19'] - o3_mean) / 10
    T_20 = (T_sample['max_o3_20'] - o3_mean) / 10

    # stack
    T_vars = np.vstack((T_01, T_02, T_03, T_04, T_05,
                        T_06, T_07, T_08, T_09, T_10,
                        T_11, T_12, T_13, T_14, T_15,
                        T_16, T_17, T_18, T_19, T_20))

    ## creating confounder coefficient

    # we're only confounding the critical exposures if the effect is < -5 (can be changed later)
    critical_to_confound = [index for index, value in enumerate(cw_window_type) if value < -0.5]

    X = [sum(fx) for fx in zip(*T_vars[critical_to_confound]*1.25)] + np.random.normal(size = n_samples, loc = 0, scale = 1)
    # z scale
    X = X / X.std()
    X = pd.Series(X)
    X = X.to_numpy()

    ## creating the outcome

    # the coefficients and treatment variables
    tx_fx, tx = cw_window_type, T_vars
    # the vectors of each coefficient * treatment variable
    tx_fx_list = [tx[i] * tx_fx[i] for i in np.arange(0, 20)]
    # tx_fx_list = [tx[10] * tx_fx[10]]
    # total treatment effect by individual
    total_tx_fx = [sum(fx) for fx in zip(*tx_fx_list)]

    # the confounder effect (z-scaled, see above)
    b_W0y = 1.5
    # the vector of confounder effects
    wx_fx = X * b_W0y

    # bit of noise
    e = np.random.normal(size=n_samples, loc = 0, scale = 1)

    # interaction
    b_intx = -12
    intx_fx = b_intx * tx[8] * tx[9] #[b_intx * tx[i] for i in np.arange(8, 10)]

    b_int = bwp_mean

    y = b_int + total_tx_fx + wx_fx + e + intx_fx

    sim_dat = pd.DataFrame({'sim_index': iteration, 'cw_size': window_size,
                            'cw_timedep': window_time, #"critical": critical_or_not,
                            'total_fx_size': effect_size,
                            'y_hat': y, 'true_y': T_sample["birthweightgrams"], 'x': X,
                            'tx_01': T_01, 'tx_02': T_02, 'tx_03': T_03, 'tx_04': T_04, 
                            'tx_05': T_05, 'tx_06': T_06, 'tx_07': T_07, 'tx_08': T_08, 
                            'tx_09': T_09, 'tx_10': T_10, 'tx_11': T_11, 'tx_12': T_12, 
                            'tx_13': T_13, 'tx_14': T_14, 'tx_15': T_15, 'tx_16': T_16, 
                            'tx_17': T_17, 'tx_18': T_18, 'tx_19': T_19, 'tx_20': T_20,
                            'tx_9_10': T_09*T_10})


    #### Part II: model training
    # dir = "data/sims/" + "sim" + str(iteration).zfill(3) + "_" + window_size + "_" + window_time + ".csv"
    # sim_dat = pd.read_csv(dir)

    T = sim_dat.loc[:, "tx_01":"tx_9_10"]
    X = pd.DataFrame({"confounder": sim_dat["x"]})
    y = pd.DataFrame({"birthweight": sim_dat["y_hat"]})
    # convert to 1d array
    # y = y.to_numpy().flatten()

    ## for interactions

    # featurizer = PolynomialFeatures(degree = 2, interaction_only = True, include_bias = False, order = 'F')

    # T_interactions = featurizer.fit_transform(T)

    print("test0")

    est = CausalForestDML(model_t='forest',
                          model_y='forest',
                          discrete_treatment=False,
                          n_estimators=500,
                          n_jobs = 4)

    est.fit(y, T=T, X=X, W=X)

    print("test1")

    # # extract results
    treatments = np.array(est.cate_treatment_names())
    res = est.marginal_ate_inference(T, X)
    means = res.mean_point #est.marginal_ate(T, X)
    ci_lower, ci_upper = res.conf_int_mean()
    p_vals = res.pvalue()

    # print(res)
    # print(len(res))

    # # create dataframe
    res_df = pd.DataFrame({
        # 'sim_index': iteration,
        # 'cw_size': window_size, 'cw_timedep': window_time,
        'treatment': treatments,
        # 'true_effect': cw_window_type,
        'mean': means[0],
        'ci_lower': ci_lower[0],
        'ci_upper': ci_upper[0],
        'p_value': p_vals[0]
    })
    
    # indicate if critical
    # res_df["cw"] = ["critical" if x < -0.5 else "not critical" for x in res_df["true_effect"]]

    ## test results for critical window

    # 1. is the true effect recovered for each week?
    # res_df["effect_recovered"] = res_df["true_effect"].between(res_df["ci_lower"], res_df["ci_upper"])

    # 2. did it get the trend? i.e., the difference in value over time
    # what the difference should be:
    # diff_list_true = [np.nan]
    # for i in range(1, len(cw_window_type)):
    #     diff_list_true.append(cw_window_type[i] - cw_window_type[i - 1])
    # res_df["trend_true"] = diff_list_true

    # diff_list_pred = [np.nan]
    # for i in range(1, len(res_df["mean"])):
    #     diff_list_pred.append(res_df.loc[i, "mean"] - res_df.loc[i - 1, "mean"])
    # res_df["trend_pred"] = diff_list_pred

    # # how close is the true trend to the observed trend?
    # res_df["trend_recovered_diff"] = (res_df["trend_true"] - res_df["trend_pred"])

    # # 3. did it get the peak effect? i.e., the inflection point
    # # what the inflection point should be:
    # inflection = []
    # for i in range(len(diff_list_true) - 1):
    #     if diff_list_true[i] < 0 and diff_list_true[i + 1] > 0:
    #         inflection.append("peak")
    #     else:
    #         inflection.append("not peak")
    # inflection.append(np.nan) # no next value
    # res_df["inflection_true"] = inflection

    # # what was the predicted inflection point?
    # inflection = []
    # for i in range(len(diff_list_pred) - 1):
    #     if diff_list_pred[i] < 0 and diff_list_pred[i + 1] > 0:
    #         inflection.append("peak")
    #     else:
    #         inflection.append("not peak")
    # inflection.append(np.nan) # no next value
    # res_df["inflection_pred"] = inflection

    # # is the peak effect recovered?
    # res_df["inflection_recovered"] = (res_df["inflection_true"] == res_df["inflection_pred"])

    # # 4. was the critical effect (>5) statistically significant at alpha = 0.05?
    # critical_sig = []

    # for index, item in enumerate(res_df["p_value"]):
    #     if index in [index for index, value in enumerate(cw_window_type) if value < -0.5]:
    #         if item < 0.05:
    #             critical_sig.append("TRUE")
    #         else:
    #             critical_sig.append("FALSE")
    #     else:
    #         critical_sig.append(np.nan)

    # res_df["critical_sig_recovered"] = critical_sig

    res_df.to_csv("data/intx_sim_results/" + "sim" + str(iteration).zfill(3) + "_res_intx" + ".csv", sep = ',', index = False)

In [22]:
# n_samples = 5000

# n_X = 1
# n_T = 20
# n_W = 1

#     # treatments / exposures; 10 ppb scale
# T_sample = births.sample(n = n_samples, replace = False)

# T_01 = (T_sample['max_o3_01'] - o3_mean) / 10
# T_03 = (T_sample['max_o3_03'] - o3_mean) / 10
# T_02 = (T_sample['max_o3_02'] - o3_mean) / 10
# T_04 = (T_sample['max_o3_04'] - o3_mean) / 10
# T_05 = (T_sample['max_o3_05'] - o3_mean) / 10
# T_06 = (T_sample['max_o3_06'] - o3_mean) / 10
# T_07 = (T_sample['max_o3_07'] - o3_mean) / 10
# T_08 = (T_sample['max_o3_08'] - o3_mean) / 10
# T_09 = (T_sample['max_o3_09'] - o3_mean) / 10
# T_10 = (T_sample['max_o3_10'] - o3_mean) / 10
# T_11 = (T_sample['max_o3_11'] - o3_mean) / 10
# T_12 = (T_sample['max_o3_12'] - o3_mean) / 10
# T_13 = (T_sample['max_o3_13'] - o3_mean) / 10
# T_14 = (T_sample['max_o3_14'] - o3_mean) / 10
# T_15 = (T_sample['max_o3_15'] - o3_mean) / 10
# T_16 = (T_sample['max_o3_16'] - o3_mean) / 10
# T_17 = (T_sample['max_o3_17'] - o3_mean) / 10
# T_18 = (T_sample['max_o3_18'] - o3_mean) / 10
# T_19 = (T_sample['max_o3_19'] - o3_mean) / 10
# T_20 = (T_sample['max_o3_20'] - o3_mean) / 10

#     # stack
# T_vars = np.vstack((T_01, T_02, T_03, T_04, T_05,
#                     T_06, T_07, T_08, T_09, T_10,
#                     T_11, T_12, T_13, T_14, T_15,
#                     T_16, T_17, T_18, T_19, T_20))

#     ## creating confounder coefficient

#     # we're only confounding the critical exposures if the effect is < -5 (can be changed later)
# critical_to_confound = [index for index, value in enumerate(norm_moderate_fx) if value < -0.5]

# X = [sum(fx) for fx in zip(*T_vars[critical_to_confound]*1.25)] + np.random.normal(size = n_samples, loc = 0, scale = 1)
#     # z scale
# X = X / X.std()
# X = pd.Series(X)
# X = X.to_numpy()

#     ## creating the outcome

#     # the coefficients and treatment variables
# tx_fx, tx = norm_moderate_fx, T_vars
#     # the vectors of each coefficient * treatment variable
# tx_fx_list = [tx[i] * tx_fx[i] for i in np.arange(0, 20)]
#     # tx_fx_list = [tx[10] * tx_fx[10]]
#     # total treatment effect by individual
# total_tx_fx = [sum(fx) for fx in zip(*tx_fx_list)]

#     # the confounder effect (z-scaled, see above)
# b_W0y = 1.5
#     # the vector of confounder effects
# wx_fx = X * b_W0y

#     # bit of noise
# e = np.random.normal(size=n_samples, loc = 0, scale = 1)

#     # interaction
# b_intx = -12
# intx_fx = b_intx * tx[8] * tx[9] #[b_intx * tx[i] for i in np.arange(8, 10)]

# b_int = bwp_mean

# y = b_int + total_tx_fx + wx_fx + e + intx_fx

# sim_dat = pd.DataFrame({'sim_index': 1,# 'cw_size': window_size,
#                         #'cw_timedep': window_time, #"critical": critical_or_not,
#                         'total_fx_size': effect_size,
#                         'y_hat': y, 'true_y': T_sample["birthweightgrams"], 'x': X,
#                         'tx_01': T_01, 'tx_02': T_02, 'tx_03': T_03, 'tx_04': T_04, 
#                         'tx_05': T_05, 'tx_06': T_06, 'tx_07': T_07, 'tx_08': T_08, 
#                         'tx_09': T_09, 'tx_10': T_10, 'tx_11': T_11, 'tx_12': T_12, 
#                         'tx_13': T_13, 'tx_14': T_14, 'tx_15': T_15, 'tx_16': T_16, 
#                         'tx_17': T_17, 'tx_18': T_18, 'tx_19': T_19, 'tx_20': T_20,
#                         'tx_9_10': T_09*T_10})


#     #### Part II: model training
#     # dir = "data/sims/" + "sim" + str(iteration).zfill(3) + "_" + window_size + "_" + window_time + ".csv"
#     # sim_dat = pd.read_csv(dir)

# T = sim_dat.loc[:, "tx_01":"tx_9_10"]
# X = pd.DataFrame({"confounder": sim_dat["x"]})
# y = pd.DataFrame({"birthweight": sim_dat["y_hat"]})

# est = CausalForestDML(model_t='forest',
#                           model_y='forest',
#                           discrete_treatment=False,
#                           n_estimators=500,
#                           n_jobs = 4)

# est.fit(y, T=T, X=X, W=X)


# res_df = pd.DataFrame({
#     'sim_index': iteration,
#     # 'cw_size': window_size, 'cw_timedep': window_time,
#     'treatment': treatments,
#     'true_effect': cw_window_type,
#     'mean': means[0],
#     'ci_lower': ci_lower[0],
#     'ci_upper': ci_upper[0],
#     'p_value': p_vals[0]
# })

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/base.py:1389: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/base.py:1389: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/base.py:1389: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklea

In [48]:
# treatments = np.array(est.cate_treatment_names())
# res = est.marginal_ate_inference(T, X)
# means = res.mean_point #est.marginal_ate(T, X)
# ci_lower, ci_upper = res.conf_int_mean()
# p_vals = res.pvalue()

# res_df = pd.DataFrame({
#     'sim_index': 1,
#     # 'cw_size': window_size, 'cw_timedep': window_time,
#     'treatment': treatments,
#     # 'true_effect': cw_window_type,
#     'mean': means[0],
#     'ci_lower': ci_lower[0],
#     'ci_upper': ci_upper[0],
#     'p_value': p_vals[0]
# })

In [50]:
nsims = 600

# [dml_function(x) for x in range(1, nsims + 1)]
# dml_function(1)

# ## all cw structures
# for i in range(0, len(cw_combos)):
for x in range(1, nsims + 1):
    dml_function(x, cw_combos.loc[0, "coefficients"],
                 cw_combos.loc[0, "times"],
                 cw_combos.loc[0, "sizes"])
        # print(x, cw_combos.loc[i, "sizes"], 
        #         cw_combos.loc[i, "times"])

### missed one — naive narrow
# for x in range(1, nsims + 1):
#     dml_function(x, cw_combos.loc[1, "coefficients"], 
#                     cw_combos.loc[1, "sizes"], 
#                     cw_combos.loc[1, "times"])

test0


/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/base.py:1389: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/base.py:1389: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/base.py:1389: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklea

test1
   treatment       mean   ci_lower  ci_upper       p_value
0      tx_01  -0.022644  -1.256859  1.211570  9.713146e-01
1      tx_02  -0.046218  -1.110740  1.018304  9.321859e-01
2      tx_03   0.013331  -1.069396  1.096058  9.807476e-01
3      tx_04  -0.037077  -1.045732  0.971578  9.425655e-01
4      tx_05  -0.066816  -1.049222  0.915591  8.939551e-01
5      tx_06   0.152464  -0.838360  1.143287  7.629639e-01
6      tx_07  -0.158991  -1.197739  0.879758  7.641831e-01
7      tx_08  -0.791746  -1.924568  0.341077  1.707347e-01
8      tx_09  -3.539834  -6.279046 -0.800623  1.131469e-02
9      tx_10  -6.018129  -8.627134 -3.409124  6.154790e-06
10     tx_11  -3.715517  -4.806426 -2.624608  2.465201e-11
11     tx_12  -0.711815  -1.789397  0.365766  1.954278e-01
12     tx_13  -0.071172  -1.020523  0.878178  8.831815e-01
13     tx_14  -0.186592  -1.194161  0.820978  7.166307e-01
14     tx_15   0.097126  -0.971742  1.165994  8.586460e-01
15     tx_16  -0.002838  -1.036344  1.030667  9.95

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/base.py:1389: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/base.py:1389: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/base.py:1389: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklea

test1
   treatment       mean   ci_lower  ci_upper       p_value
0      tx_01  -0.004875  -1.072417  1.062667  9.928584e-01
1      tx_02   0.023726  -0.951933  0.999385  9.619851e-01
2      tx_03  -0.001481  -0.918259  0.915297  9.974741e-01
3      tx_04   0.007839  -0.925327  0.941004  9.868643e-01
4      tx_05  -0.072913  -1.023618  0.877791  8.805144e-01
5      tx_06   0.126790  -0.832182  1.085761  7.955306e-01
6      tx_07  -0.124524  -1.163220  0.914173  8.142323e-01
7      tx_08  -0.813106  -1.854326  0.228114  1.258762e-01
8      tx_09  -3.706061  -6.192283 -1.219839  3.482388e-03
9      tx_10  -6.300510  -8.883967 -3.717053  1.753469e-06
10     tx_11  -3.653025  -4.741730 -2.564319  4.818703e-11
11     tx_12  -0.777021  -1.820647  0.266605  1.444905e-01
12     tx_13   0.022563  -0.955207  1.000334  9.639250e-01
13     tx_14  -0.017249  -0.916045  0.881547  9.699952e-01
14     tx_15  -0.120834  -1.057159  0.815492  8.003184e-01
15     tx_16  -0.036127  -0.975645  0.903392  9.39

...

test1
   treatment       mean   ci_lower  ci_upper       p_value
0      tx_01   0.036602  -1.151579  1.224783  9.518554e-01
1      tx_02  -0.150500  -1.278011  0.977012  7.936184e-01
2      tx_03   0.155935  -0.933291  1.245162  7.790241e-01
3      tx_04   0.091518  -0.854473  1.037508  8.496132e-01
4      tx_05  -0.042255  -1.140699  1.056189  9.398995e-01
5      tx_06  -0.045578  -1.022409  0.931253  9.271353e-01
6      tx_07   0.011858  -1.001685  1.025401  9.817049e-01
7      tx_08  -0.809434  -1.947601  0.328733  1.633558e-01
8      tx_09  -3.553422  -6.550966 -0.555879  2.015612e-02
9      tx_10  -5.862177  -8.846753 -2.877602  1.182730e-04
10     tx_11  -3.545429  -4.706179 -2.384679  2.143119e-09
11     tx_12  -0.907186  -1.977932  0.163559  9.679905e-02
12     tx_13  -0.145077  -1.113902  0.823747  7.691428e-01
13     tx_14   0.166996  -0.812559  1.146552  7.382752e-01
14     tx_15  -0.064886  -1.168670  1.038899  9.082741e-01
15     tx_16   0.081887  -0.917236  1.081011  8.72

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/base.py:1389: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/base.py:1389: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/base.py:1389: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklea

test1
   treatment       mean   ci_lower  ci_upper       p_value
0      tx_01   0.103714  -1.072196  1.279624  8.627559e-01
1      tx_02  -0.015798  -1.133605  1.102009  9.779011e-01
2      tx_03   0.032896  -0.961018  1.026810  9.482779e-01
3      tx_04  -0.024889  -0.985618  0.935841  9.595051e-01
4      tx_05   0.041837  -0.929483  1.013156  9.327226e-01
5      tx_06  -0.009453  -0.997911  0.979004  9.850446e-01
6      tx_07  -0.098138  -1.098130  0.901853  8.474691e-01
7      tx_08  -0.846511  -1.936004  0.242982  1.277972e-01
8      tx_09  -3.357359  -6.487404 -0.227315  3.552671e-02
9      tx_10  -5.565561  -8.691304 -2.439818  4.833345e-04
10     tx_11  -3.665525  -4.776876 -2.554174  1.016531e-10
11     tx_12  -0.800973  -1.893700  0.291755  1.508147e-01
12     tx_13  -0.162534  -1.126431  0.801362  7.410272e-01
13     tx_14   0.111442  -0.862015  1.084898  8.224636e-01
14     tx_15  -0.034885  -1.066320  0.996550  9.471474e-01
15     tx_16  -0.154036  -1.165868  0.857797  7.65

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/base.py:1389: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/base.py:1389: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklearn/base.py:1389: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return fit_method(estimator, *args, **kwargs)
/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/sklea

test1
   treatment       mean   ci_lower  ci_upper       p_value
0      tx_01   0.071836  -1.012446  1.156117  8.966841e-01
1      tx_02  -0.120101  -1.103095  0.862894  8.107446e-01
2      tx_03   0.074555  -0.937748  1.086857  8.852253e-01
3      tx_04  -0.017847  -1.101717  1.066024  9.742551e-01
4      tx_05   0.004221  -0.944001  0.952443  9.930394e-01
5      tx_06  -0.044518  -1.038243  0.949207  9.300323e-01
6      tx_07  -0.027821  -0.985599  0.929957  9.546000e-01
7      tx_08  -0.831936  -1.867477  0.203606  1.153487e-01
8      tx_09  -3.665079  -6.265183 -1.064974  5.731781e-03
9      tx_10  -6.075323  -8.754146 -3.396501  8.788445e-06
10     tx_11  -3.563576  -4.631572 -2.495580  6.160045e-11
11     tx_12  -0.892695  -1.925715  0.140324  9.031762e-02
12     tx_13   0.032783  -1.059739  1.125305  9.531022e-01
13     tx_14  -0.023335  -1.044504  0.997835  9.642771e-01
14     tx_15  -0.044766  -1.003899  0.914367  9.271124e-01
15     tx_16   0.071303  -0.905367  1.047974  8.86

In [ ]:
# # test = est.marginal_ate_inference(T, X)
# # test.shape()
# # np.array(test.pvalue())
# # test.pvalue()
# # est.marginal_ate_inference(T, X)

# # res.mean_point
# norm_moderate_fx

In [ ]:



# [item for item in norm_moderate_fx if item < -5]

# fx = ["T_" + str(num) for num in 
#  [index for index, value in enumerate(norm_moderate_fx) if value < -5]]

# # fx[0] * 2.5

# # [item for item in T_vars if item in fx]
# any(item in T_vars for item in fx)

# # for i in fx:
# #     print([item for item in T_vars if i in item])
# # [item for item in my_list if search_string in item]
    
# T_vars[critical_to_confound] * 2.5
# [sum(fx) for fx in zip(*T_vars[critical_to_confound]*2.5)] + np.random.normal(size = n_samples, loc = 0, scale = 1)
# test = b_W0T9 * T_9 + b_W0T10 * T_10 + b_W0T11 * T_11 + np.random.normal(size=n_samples)
# type(test)
